# PlayStation 4 Games Sales Analysis

This notebook contains a comprehensive analysis of PlayStation 4 game sales across different regions and years.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")
%matplotlib inline

## Data Loading

First, we'll load the PS4 sales data from the raw data directory.

In [ ]:
# Path to the data file
data_path = '../data/raw/ps4_sales.csv'

# Load the data
df = pd.read_csv(data_path)

# Display the first few rows
df.head()